In [12]:
from groq import Groq
import json
import csv

# تنظیمات پیام‌های سیستم و کاربر
system_message = """شما یک دستیار مفید هستید که جفت‌های تاریخ فارسی (غیررسمی و رسمی) تولید می‌کند. 
شما به ماه‌های تقویم جلالی مسلط هستید و می‌دانید که ماه فروردین ماه اول است. 
لطفاً تاریخ‌ها را به صورت دقیق و با تنوع زیاد در نحوه نمایش تولید کنید."""

user_message = """لطفاً 2000 جفت تاریخ به فرمت‌های زیر تولید کنید:

1. تاریخ غیررسمی به فارسی (با تنوع در نحوه نمایش)
2. همان تاریخ به فرمت استاندارد "YYYY/MM/DD"

برای تاریخ غیررسمی، از انواع مختلف نمایش استفاده کنید، از جمله:
- ترتیب معمول فارسی: "1 مهر 1403" یا "اول مهر سال 1403"
- ترتیب عددی: "1 1403 مهر"
- ترتیب با کلمه سال در ابتدا: "سال مهر اول 1403"
- استفاده از خط تیره: "1403-07-01"

برای تاریخ رسمی، همیشه از فرمت "YYYY/MM/DD" استفاده کنید، حتی اگر ورودی با خط تیره باشد.

لطفاً از انواع مختلف ماه‌های شمسی و روزهای مختلف استفاده کنید. تاریخ‌ها را به صورت تصادفی انتخاب کنید و از سال‌های مختلف (بین 1300 تا 1400) استفاده نمایید.

لطفاً خروجی را به صورت یک لیست از جفت‌های تاریخ در فرمت JSON ارائه دهید. مثال:

[
  {"informal": "1403-07-01", "formal": "1403/07/01"},
  {"informal": "پنجم اردیبهشت 1402", "formal": "1402/02/05"},
  {"informal": "17 آبان 1395", "formal": "1395/08/17"},
  {"informal": "1 1403 مهر", "formal": "1403/07/01"},
  {"informal": "سال مهر اول 1403", "formal": "1403/07/01"},
  ...
]
فقط حاوی فایل حیسون باشد و هیج متن اضافه ای در خروجی نباشد
خروجی به صورت فایل jsonتولید کن"""

client = Groq(api_key="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")

def preprocess_output(output):
    last_index = output.rindex('informal')
    start = output.find('[')
    list_str = output[start:last_index-6]+']'
    return list_str

# فایل CSV خروجی
csv_filename = 'date.csv'

# اجرای کد 20 بار
for i in range(200):
    print(f"اجرا شماره {i+1}")

    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0.4,
        max_tokens=4000,
        top_p=1,
        stream=False,
        stop=None,
    )

    output = completion.choices[0].message.content.strip()
    processed_output = preprocess_output(output)

    if processed_output:
        try:
            date_pairs = json.loads(processed_output)
            print(f"تبدیل به JSON موفقیت‌آمیز بود. تعداد جفت‌های تاریخ: {len(date_pairs)}")
            
            # ذخیره نتایج در فایل CSV با حالت append
            with open(csv_filename, 'a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                
                for pair in date_pairs:
                    writer.writerow([pair['informal'], pair['formal']])
            
            print(f"داده‌ها در فایل {csv_filename} ذخیره شدند.")
        except json.JSONDecodeError as e:
            print(f"خطا در تبدیل به JSON: {e}")
            print("لطفاً خروجی پیش‌پردازش شده را بررسی کنید:")
            print(processed_output)
    else:
        print("خطا در پیش‌پردازش خروجی. آرایه JSON یافت نشد.")


اجرا شماره 1


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01ja14mh20ecqb4f5gwjkwa1bp` on : Limit 500000, Used 501840, Requested 527. Please try again in 6m49.1848s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}